Unduh data korpus Wikipedia Bahasa Indonesia melalui link berikut https://dumps.wikimedia.org/idwiki/latest/. Cari file dengan nama “idwiki-latest-pages-articles.xml.bz2”. Data korpus setiap waktunya mengalami peningkatan. Sekitar agustus 2019 terdapat 406855 kosa kata dengan ukuran 512 MB.

Data korpus tersebut masih menggunakan format XML sehingga perlu di olah ke dalam bentuk teks. library Gensim sudah menyediakan teknik pra poses ini. Untuk mengkonversi format tersebut dapat menggunakan kode berikut:

In [1]:
from __future__ import print_function
 
# Ignore warnings dari gensim
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
 
import logging
import os.path
import sys
 
from gensim.corpora import WikiCorpus
 
program = os.path.basename(sys.argv[0])
logger = logging.getLogger(program)
 
logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
logging.root.setLevel(level=logging.INFO)
logger.info("running %s" % ' '.join(sys.argv))
 
namaFileInput = "idwiki-latest-pages-articles.xml.bz2"
namaFileOutput = "wiki.id.text"
 
space = " "
i = 0
 
# Write file ke variabel namaFileOutput encoder utf-8
output = open(namaFileOutput, 'w', encoding='utf-8')
 
# lower=False: huruf kecil dan besar dibedakan
wiki = WikiCorpus(namaFileInput, lemmatize=None, dictionary={}, lower=False)
for text in wiki.get_texts():
    output.write(' '.join(text) + '\n')
    i = i + 1
    if i % 10000 == 0:
        logger.info("Saved " + str(i) + " articles")
 
output.close()
logger.info("Finished Saved " + str(i) + " articles")

2023-10-31 21:23:40,253: INFO: running C:\Users\mmahf\miniconda3\envs\latihanstki\lib\site-packages\ipykernel_launcher.py -f C:\Users\mmahf\AppData\Roaming\jupyter\runtime\kernel-f463b008-85a3-46c7-8ba9-86ed15932aa1.json
2023-10-31 21:24:41,178: INFO: Saved 10000 articles
2023-10-31 21:25:24,173: INFO: Saved 20000 articles
2023-10-31 21:25:59,738: INFO: Saved 30000 articles
2023-10-31 21:26:30,676: INFO: Saved 40000 articles
2023-10-31 21:27:03,921: INFO: Saved 50000 articles
2023-10-31 21:27:34,986: INFO: Saved 60000 articles
2023-10-31 21:28:05,114: INFO: Saved 70000 articles
2023-10-31 21:28:37,867: INFO: Saved 80000 articles
2023-10-31 21:29:22,259: INFO: Saved 90000 articles
2023-10-31 21:29:56,929: INFO: Saved 100000 articles
2023-10-31 21:30:29,793: INFO: Saved 110000 articles
2023-10-31 21:30:58,735: INFO: Saved 120000 articles
2023-10-31 21:31:38,478: INFO: Saved 130000 articles
2023-10-31 21:32:11,733: INFO: Saved 140000 articles
2023-10-31 21:32:34,976: INFO: Saved 150000 ar

Dari kode di atas akan menghasilkan output file dari rangkaian beberapa artikel, di mana satu barisnya mewakili satu artikel.

Setelah data berhasil terkonversi, selanjutnya dilakukan training Word2Vec menggunakan kode berikut:

In [2]:
import multiprocessing
import logging
import os.path
import sys
import multiprocessing
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
 
program = os.path.basename(sys.argv[0])
logger = logging.getLogger(program)
 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
logging.root.setLevel(level=logging.INFO)
logger.info("running %s" % ' '.join(sys.argv))
 
namaFileInput = "wiki.id.text"
namaFileOutput = "w2vec_wiki_id300_0.txt"
 
 
# size 300 = 300 dimensi vektor, window 10 = 10 pengaruh kata disekitarnya, min_count 5 = kata-kata yang muncul < 5 kali akan dikeluarkan dari kosakata dan diabaikan selama pelatihan, sg 0 = cbow / sg 1 = skip gram 
model = Word2Vec(LineSentence(namaFileInput), vector_size=300, window=10, min_count=5, sg=0, workers=multiprocessing.cpu_count())
 
# trim unneeded model memory = use (much) less RAM
model.init_sims(replace=True)
model.wv.save_word2vec_format(namaFileOutput, binary=False)

2023-10-31 21:49:00,353: INFO: running C:\Users\mmahf\miniconda3\envs\latihanstki\lib\site-packages\ipykernel_launcher.py -f C:\Users\mmahf\AppData\Roaming\jupyter\runtime\kernel-f463b008-85a3-46c7-8ba9-86ed15932aa1.json
2023-10-31 21:49:00,360: INFO: collecting all words and their counts
2023-10-31 21:49:00,377: INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-10-31 21:49:04,948: INFO: PROGRESS: at sentence #10000, processed 10738098 words, keeping 353874 word types
2023-10-31 21:49:07,606: INFO: PROGRESS: at sentence #20000, processed 17334952 words, keeping 490368 word types
2023-10-31 21:49:09,567: INFO: PROGRESS: at sentence #30000, processed 22148080 words, keeping 591133 word types
2023-10-31 21:49:11,457: INFO: PROGRESS: at sentence #40000, processed 26549341 words, keeping 676278 word types
2023-10-31 21:49:13,525: INFO: PROGRESS: at sentence #50000, processed 31270076 words, keeping 775169 word types
2023-10-31 21:49:15,274: INFO: PROGRESS: at sent

2023-10-31 21:50:29,601: INFO: EPOCH 0 - PROGRESS: at 0.04% examples, 457247 words/s, in_qsize 15, out_qsize 0
2023-10-31 21:50:30,613: INFO: EPOCH 0 - PROGRESS: at 0.14% examples, 511919 words/s, in_qsize 11, out_qsize 0
2023-10-31 21:50:31,620: INFO: EPOCH 0 - PROGRESS: at 0.22% examples, 527769 words/s, in_qsize 11, out_qsize 0
2023-10-31 21:50:32,635: INFO: EPOCH 0 - PROGRESS: at 0.28% examples, 537464 words/s, in_qsize 11, out_qsize 1
2023-10-31 21:50:33,648: INFO: EPOCH 0 - PROGRESS: at 0.39% examples, 545087 words/s, in_qsize 11, out_qsize 0
2023-10-31 21:50:34,650: INFO: EPOCH 0 - PROGRESS: at 0.50% examples, 549519 words/s, in_qsize 11, out_qsize 1
2023-10-31 21:50:35,658: INFO: EPOCH 0 - PROGRESS: at 0.67% examples, 556263 words/s, in_qsize 7, out_qsize 1
2023-10-31 21:50:36,661: INFO: EPOCH 0 - PROGRESS: at 0.77% examples, 558461 words/s, in_qsize 12, out_qsize 0
2023-10-31 21:50:37,692: INFO: EPOCH 0 - PROGRESS: at 0.87% examples, 562693 words/s, in_qsize 3, out_qsize 0
202

2023-10-31 21:51:45,562: INFO: EPOCH 0 - PROGRESS: at 20.96% examples, 596565 words/s, in_qsize 0, out_qsize 0
2023-10-31 21:51:46,562: INFO: EPOCH 0 - PROGRESS: at 21.30% examples, 596036 words/s, in_qsize 8, out_qsize 1
2023-10-31 21:51:47,586: INFO: EPOCH 0 - PROGRESS: at 21.71% examples, 596426 words/s, in_qsize 0, out_qsize 0
2023-10-31 21:51:48,610: INFO: EPOCH 0 - PROGRESS: at 22.06% examples, 596380 words/s, in_qsize 2, out_qsize 0
2023-10-31 21:51:49,625: INFO: EPOCH 0 - PROGRESS: at 22.41% examples, 596306 words/s, in_qsize 0, out_qsize 0
2023-10-31 21:51:50,640: INFO: EPOCH 0 - PROGRESS: at 22.79% examples, 596058 words/s, in_qsize 0, out_qsize 0
2023-10-31 21:51:51,644: INFO: EPOCH 0 - PROGRESS: at 23.24% examples, 596085 words/s, in_qsize 0, out_qsize 0
2023-10-31 21:51:52,650: INFO: EPOCH 0 - PROGRESS: at 23.72% examples, 596393 words/s, in_qsize 0, out_qsize 1
2023-10-31 21:51:53,663: INFO: EPOCH 0 - PROGRESS: at 24.07% examples, 596162 words/s, in_qsize 0, out_qsize 1
2

2023-10-31 21:53:00,582: INFO: EPOCH 0 - PROGRESS: at 66.59% examples, 590914 words/s, in_qsize 0, out_qsize 0
2023-10-31 21:53:01,583: INFO: EPOCH 0 - PROGRESS: at 66.96% examples, 590825 words/s, in_qsize 0, out_qsize 0
2023-10-31 21:53:02,623: INFO: EPOCH 0 - PROGRESS: at 67.35% examples, 590465 words/s, in_qsize 10, out_qsize 1
2023-10-31 21:53:03,627: INFO: EPOCH 0 - PROGRESS: at 67.89% examples, 590616 words/s, in_qsize 0, out_qsize 0
2023-10-31 21:53:04,627: INFO: EPOCH 0 - PROGRESS: at 68.36% examples, 590267 words/s, in_qsize 0, out_qsize 0
2023-10-31 21:53:05,644: INFO: EPOCH 0 - PROGRESS: at 68.81% examples, 590036 words/s, in_qsize 0, out_qsize 0
2023-10-31 21:53:06,667: INFO: EPOCH 0 - PROGRESS: at 69.28% examples, 589861 words/s, in_qsize 0, out_qsize 0
2023-10-31 21:53:07,672: INFO: EPOCH 0 - PROGRESS: at 69.76% examples, 589842 words/s, in_qsize 0, out_qsize 0
2023-10-31 21:53:08,675: INFO: EPOCH 0 - PROGRESS: at 70.25% examples, 589758 words/s, in_qsize 0, out_qsize 0


2023-10-31 21:54:15,476: INFO: EPOCH 0 - PROGRESS: at 99.71% examples, 586882 words/s, in_qsize 0, out_qsize 1
2023-10-31 21:54:16,168: INFO: EPOCH 0: training on 149788691 raw words (133568699 effective words) took 227.6s, 586876 effective words/s
2023-10-31 21:54:17,177: INFO: EPOCH 1 - PROGRESS: at 0.05% examples, 541493 words/s, in_qsize 14, out_qsize 1
2023-10-31 21:54:18,194: INFO: EPOCH 1 - PROGRESS: at 0.16% examples, 563670 words/s, in_qsize 11, out_qsize 0
2023-10-31 21:54:19,220: INFO: EPOCH 1 - PROGRESS: at 0.23% examples, 549738 words/s, in_qsize 16, out_qsize 2
2023-10-31 21:54:20,224: INFO: EPOCH 1 - PROGRESS: at 0.28% examples, 538841 words/s, in_qsize 11, out_qsize 0
2023-10-31 21:54:21,242: INFO: EPOCH 1 - PROGRESS: at 0.39% examples, 547596 words/s, in_qsize 10, out_qsize 0
2023-10-31 21:54:22,246: INFO: EPOCH 1 - PROGRESS: at 0.51% examples, 553643 words/s, in_qsize 12, out_qsize 1
2023-10-31 21:54:23,247: INFO: EPOCH 1 - PROGRESS: at 0.67% examples, 561655 words/s,

...

2023-10-31 22:04:14,730: INFO: EPOCH 3 - PROGRESS: at 58.99% examples, 583147 words/s, in_qsize 15, out_qsize 0
2023-10-31 22:04:15,732: INFO: EPOCH 3 - PROGRESS: at 59.44% examples, 583432 words/s, in_qsize 0, out_qsize 0
2023-10-31 22:04:16,748: INFO: EPOCH 3 - PROGRESS: at 59.84% examples, 583394 words/s, in_qsize 0, out_qsize 0
2023-10-31 22:04:17,748: INFO: EPOCH 3 - PROGRESS: at 60.24% examples, 583333 words/s, in_qsize 0, out_qsize 0
2023-10-31 22:04:18,763: INFO: EPOCH 3 - PROGRESS: at 60.64% examples, 583116 words/s, in_qsize 9, out_qsize 0
2023-10-31 22:04:19,770: INFO: EPOCH 3 - PROGRESS: at 61.06% examples, 583334 words/s, in_qsize 0, out_qsize 0
2023-10-31 22:04:20,775: INFO: EPOCH 3 - PROGRESS: at 61.44% examples, 583341 words/s, in_qsize 0, out_qsize 1
2023-10-31 22:04:21,783: INFO: EPOCH 3 - PROGRESS: at 61.83% examples, 583245 words/s, in_qsize 0, out_qsize 0
2023-10-31 22:04:22,783: INFO: EPOCH 3 - PROGRESS: at 62.10% examples, 583089 words/s, in_qsize 1, out_qsize 0


2023-10-31 22:05:29,730: INFO: EPOCH 3 - PROGRESS: at 91.81% examples, 582208 words/s, in_qsize 15, out_qsize 0
2023-10-31 22:05:30,751: INFO: EPOCH 3 - PROGRESS: at 92.16% examples, 582037 words/s, in_qsize 10, out_qsize 1
2023-10-31 22:05:31,784: INFO: EPOCH 3 - PROGRESS: at 92.46% examples, 581767 words/s, in_qsize 15, out_qsize 0
2023-10-31 22:05:32,811: INFO: EPOCH 3 - PROGRESS: at 92.77% examples, 581598 words/s, in_qsize 6, out_qsize 0
2023-10-31 22:05:33,831: INFO: EPOCH 3 - PROGRESS: at 93.13% examples, 581582 words/s, in_qsize 0, out_qsize 0
2023-10-31 22:05:34,846: INFO: EPOCH 3 - PROGRESS: at 93.48% examples, 581286 words/s, in_qsize 12, out_qsize 1
2023-10-31 22:05:35,847: INFO: EPOCH 3 - PROGRESS: at 93.77% examples, 580963 words/s, in_qsize 10, out_qsize 0
2023-10-31 22:05:36,850: INFO: EPOCH 3 - PROGRESS: at 94.06% examples, 580677 words/s, in_qsize 10, out_qsize 0
2023-10-31 22:05:37,876: INFO: EPOCH 3 - PROGRESS: at 94.37% examples, 580567 words/s, in_qsize 7, out_qsi

2023-10-31 22:06:44,701: INFO: EPOCH 4 - PROGRESS: at 9.57% examples, 501883 words/s, in_qsize 10, out_qsize 0
2023-10-31 22:06:45,714: INFO: EPOCH 4 - PROGRESS: at 9.89% examples, 503632 words/s, in_qsize 1, out_qsize 2
2023-10-31 22:06:46,715: INFO: EPOCH 4 - PROGRESS: at 10.27% examples, 505829 words/s, in_qsize 0, out_qsize 0
2023-10-31 22:06:47,733: INFO: EPOCH 4 - PROGRESS: at 10.53% examples, 505512 words/s, in_qsize 6, out_qsize 0
2023-10-31 22:06:48,734: INFO: EPOCH 4 - PROGRESS: at 10.83% examples, 506784 words/s, in_qsize 0, out_qsize 1
2023-10-31 22:06:49,762: INFO: EPOCH 4 - PROGRESS: at 11.15% examples, 507252 words/s, in_qsize 11, out_qsize 0
2023-10-31 22:06:50,766: INFO: EPOCH 4 - PROGRESS: at 11.41% examples, 506356 words/s, in_qsize 14, out_qsize 1
2023-10-31 22:06:51,782: INFO: EPOCH 4 - PROGRESS: at 11.72% examples, 506156 words/s, in_qsize 13, out_qsize 1
2023-10-31 22:06:52,783: INFO: EPOCH 4 - PROGRESS: at 11.97% examples, 505028 words/s, in_qsize 10, out_qsize 

2023-10-31 22:07:59,528: INFO: EPOCH 4 - PROGRESS: at 55.80% examples, 565839 words/s, in_qsize 0, out_qsize 0
2023-10-31 22:08:00,539: INFO: EPOCH 4 - PROGRESS: at 56.18% examples, 566225 words/s, in_qsize 0, out_qsize 0
2023-10-31 22:08:01,551: INFO: EPOCH 4 - PROGRESS: at 56.55% examples, 566569 words/s, in_qsize 0, out_qsize 0
2023-10-31 22:08:02,556: INFO: EPOCH 4 - PROGRESS: at 56.95% examples, 567117 words/s, in_qsize 0, out_qsize 0
2023-10-31 22:08:03,582: INFO: EPOCH 4 - PROGRESS: at 57.36% examples, 567421 words/s, in_qsize 0, out_qsize 0
2023-10-31 22:08:04,591: INFO: EPOCH 4 - PROGRESS: at 57.74% examples, 567839 words/s, in_qsize 0, out_qsize 1
2023-10-31 22:08:05,599: INFO: EPOCH 4 - PROGRESS: at 58.12% examples, 568090 words/s, in_qsize 0, out_qsize 0
2023-10-31 22:08:06,610: INFO: EPOCH 4 - PROGRESS: at 58.53% examples, 568480 words/s, in_qsize 0, out_qsize 0
2023-10-31 22:08:07,623: INFO: EPOCH 4 - PROGRESS: at 58.90% examples, 568883 words/s, in_qsize 0, out_qsize 0
2

2023-10-31 22:09:14,530: INFO: EPOCH 4 - PROGRESS: at 89.63% examples, 583453 words/s, in_qsize 0, out_qsize 2
2023-10-31 22:09:15,537: INFO: EPOCH 4 - PROGRESS: at 90.16% examples, 583734 words/s, in_qsize 0, out_qsize 1
2023-10-31 22:09:16,549: INFO: EPOCH 4 - PROGRESS: at 90.72% examples, 583952 words/s, in_qsize 0, out_qsize 0
2023-10-31 22:09:17,574: INFO: EPOCH 4 - PROGRESS: at 91.22% examples, 584161 words/s, in_qsize 0, out_qsize 0
2023-10-31 22:09:18,578: INFO: EPOCH 4 - PROGRESS: at 91.75% examples, 584339 words/s, in_qsize 0, out_qsize 0
2023-10-31 22:09:19,599: INFO: EPOCH 4 - PROGRESS: at 92.16% examples, 584556 words/s, in_qsize 0, out_qsize 0
2023-10-31 22:09:20,610: INFO: EPOCH 4 - PROGRESS: at 92.50% examples, 584685 words/s, in_qsize 0, out_qsize 0
2023-10-31 22:09:21,612: INFO: EPOCH 4 - PROGRESS: at 92.85% examples, 584878 words/s, in_qsize 0, out_qsize 0
2023-10-31 22:09:22,618: INFO: EPOCH 4 - PROGRESS: at 93.25% examples, 585049 words/s, in_qsize 0, out_qsize 0
2

Terdapat beberapa parameter diantaranya:

<b>vector_size</b>: Ukuran dimensi vector yang mewakili setiap token atau kata. Jika memiliki data yang terbatas, nilai size yang digunakan sebaiknya lebih kecil karena akan mempengaruhi kata unik di sekitarnya. Namun, jika memiliki banyak data maka dapat bereksperimen dengan berbagai ukuran.

<b>window</b>: Jarak maksimum antara kata target dengan kata di sekitarnya. Jika nilai window yang digunakan besar, maka terdapat banyak kata terkait disekitarnya (baik di posisi kiri dan kanan dari kata konteks). Secara teori, jika window lebih kecil akan memberikan istilah yang lebih spesifik terhadap suatu konteks kata.

<b>min_count</b>: Frekuensi minimal jumlah kata. Model akan mengabaikan kata-kata yang tidak memenuhi nilai min_count. Kata yang jarang muncul biasanya tidak terlalu penting, jadi lebih baik untuk dihilangkan. Parameter ini mungkin lebih berpengaruh pada effisiensi penggunaan memory dan ukuran file model.

<b>sg</b>: 0 untuk arsitektur Word2Vec CBOW dan 1 untuk arsitektur Word2Vec Skip-gram.

<b>workers</b>: Berapa banyak threads yang digunakan untuk melakukan multiprocessing.

Model yang sudah disimpan dapat dilakukan skenario penggunaannya sebagaimana pada kode di bawah ini untuk melihat kedekatan hubungan antara kata.

In [14]:
import gensim
#import gensim.models.keyedvectors as word2vec


namaFileModel = "w2vec_wiki_id300_0.txt"

#model = gensim.models.Word2Vec.load(namaFileModel)
#model = word2vec.KeyedVectors.load_word2vec_format(namaFileModel, binary=True)

sim = model.wv.most_similar("Bakso")
print("10 kata terdekat dari Bakso:{}".format(sim))
sim = model.wv.most_similar("Mahasiswa")
print("10 kata terdekat dari Mahasiswa:{}".format(sim))
 
sim = model.wv.similarity("Yogyakarta", "Surakarta")
print("Kedekatan Yogyakarta-Surakarta: {}".format(sim))
sim = model.wv.similarity("Yogyakarta", "Semarang")
print("Kedekatan Yogyakarta-Semarang: {}".format(sim))
 
sim = model.wv.most_similar_cosmul(positive=['minuman', 'rendang'], negative=['makanan'])
print("makanan-rendang, minuman-?: {}".format(sim))
sim = model.wv.most_similar_cosmul(positive=['mobil', 'honda'], negative=['motor'])
print("motor-honda, mobil-?: {}".format(sim))

10 kata terdekat dari Bakso:[('Rujak', 0.8258554935455322), ('Pecel', 0.8033948540687561), ('Kerupuk', 0.8005541563034058), ('Goreng', 0.7962303161621094), ('Bandeng', 0.7874205112457275), ('Bakmi', 0.7814149856567383), ('Sate', 0.7805248498916626), ('Rawon', 0.7790371179580688), ('bakso', 0.7726799249649048), ('Pindang', 0.7702051401138306)]
10 kata terdekat dari Bali:[('HMI', 0.593949556350708), ('Mahasiwa', 0.5907865762710571), ('BEM', 0.5884683132171631), ('UNISBA', 0.5834879279136658), ('UNRI', 0.5736002922058105), ('Menwa', 0.5663052797317505), ('mahasiswa', 0.5628307461738586), ('Pelajar', 0.5595340728759766), ('HMJ', 0.5539311766624451), ('kemahasiswaan', 0.5439574718475342)]
Kedekatan Yogyakarta-Surakarta: 0.7861818671226501
Kedekatan Yogyakarta-Semarang: 0.6145674586296082
makanan-rendang, minuman-?: [('kola', 0.8787573411957991), ('martabak', 0.8769399969468845), ('Fanta', 0.8754954249275457), ('Minuman', 0.8732843716921826), ('wedang', 0.8639075077541758), ('Trenz', 0.85995